In [1]:
import cv2
import dlib
import numpy as np
import os
import math

# Facial Image Archive

In [ ]:
# Create a directory to save detected faces
output_dir = 'Faces captured'
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# Initialize face detector and face descriptor
detector = dlib.cnn_face_detection_model_v1('mmod_human_face_detector.dat') # Convolutional Neural Network face detection
sp = dlib.shape_predictor('shape_predictor_68_face_landmarks.dat') # Facial landmark detection
face_encoder = dlib.face_recognition_model_v1('dlib_face_recognition_resnet_model_v1.dat') # Recognition model

def save_face(face_image, face_id):
    # Add ID to the face image and the font
    cv2.putText(face_image, f'ID: {face_id}', (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
    
    # Save the image with ID in the filename
    face_file_path = os.path.join(output_dir, f'face_{face_id}.jpg')
    cv2.imwrite(face_file_path, face_image)

def main():
    cap = cv2.VideoCapture(0)
    saved_faces = {}
    face_id = 0

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        # Resize the frame to 256x256
        frame = cv2.resize(frame, (256, 256))

        # Convert to grayscale
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        # Detect faces
        dets = detector(gray, 1)

        for det in dets:
            # Access the bounding box
            rect = det.rect
            shape = sp(frame, rect)
            face_descriptor = face_encoder.compute_face_descriptor(frame, shape)

            # Convert to numpy array for easier handling
            face_descriptor_np = np.array(face_descriptor)

            # Check if this face descriptor is similar to any already saved
            found_similar = False
            for saved_descriptor in saved_faces.values():
                if np.linalg.norm(saved_descriptor - face_descriptor_np) < 0.6:  # Threshold for similarity
                    found_similar = True
                    break

            if not found_similar:
                # Save the new face
                c=20 # Clearance
                face_image = frame[rect.top()-c:rect.bottom()+c, rect.left()-c:rect.right()+c]
                save_face(face_image, face_id)
                saved_faces[face_id] = face_descriptor_np
                face_id += 1

            # Draw rectangle around the face
            cv2.rectangle(frame, (rect.left(), rect.top()), (rect.right(), rect.bottom()), (255, 0, 0), 2)

        cv2.imshow('Face Detection', frame)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

if __name__ == '__main__':
    main()


# Facial Landmark Detection

In [ ]:
cap = cv2.VideoCapture(0)

hog_face_detector = dlib.get_frontal_face_detector()

dlib_facelandmark = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")

while True:
    _, frame = cap.read()
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    faces = hog_face_detector(gray)
    for face in faces:

        face_landmarks = dlib_facelandmark(gray, face)

        for n in range(0, 67):
            x = face_landmarks.part(n).x
            y = face_landmarks.part(n).y
            cv2.circle(frame, (x, y), 1, (0, 255, 255), 1)


    cv2.imshow("Face Landmarks", frame)

    key = cv2.waitKey(1)
    if key == 27:
        break
cap.release()
cv2.destroyAllWindows()

# Calculation of Eyes,Nose and Mouth Sizes 


In [ ]:
# Initialize video capture
cap = cv2.VideoCapture(0)

# Initialize dlib's face detector and landmark predictor
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor('shape_predictor_68_face_landmarks.dat')

def calculate_distance(point1, point2):
    return math.sqrt((point1.x - point2.x) ** 2 + (point1.y - point2.y) ** 2) # Euclidean distance

while True:
    # Capture frame-by-frame
    ret, frame = cap.read()
    if not ret:
        break

    # Convert to grayscale for face detection
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Detect faces
    faces = detector(gray)

    # Loop through detected faces
    for i, face in enumerate(faces):
        # Get landmarks
        landmarks = predictor(gray, face)

        # Draw rectangle around the face
        cv2.rectangle(frame, (face.left(), face.top()), (face.right(), face.bottom()), (255, 0, 0), 2)

        # Calculate eye sizes (landmarks 36-41 for the left eye, 42-47 for the right eye)
        left_eye_size = calculate_distance(landmarks.part(36), landmarks.part(39))  # Horizontal distance
        right_eye_size = calculate_distance(landmarks.part(42), landmarks.part(45))  # Horizontal distance

        # Calculate nose size (landmarks 30-33)
        nose_size = calculate_distance(landmarks.part(30), landmarks.part(33))

        # Calculate mouth size (landmarks 48-54)
        mouth_size = calculate_distance(landmarks.part(48), landmarks.part(54))

        # Print sizes
        print(f"Face {i}: Left Eye Size: {left_eye_size:.2f}, Right Eye Size: {right_eye_size:.2f}, Nose Size: {nose_size:.2f}, Mouth Size: {mouth_size:.2f}")

        # Draw landmarks
        for n in range(0, 67):
            x = landmarks.part(n).x
            y = landmarks.part(n).y
            cv2.circle(frame, (x, y), 3, (0, 255, 0), -1)

    # Display the resulting frame
    cv2.imshow('Video', frame)

    # Break the loop on 'q' key press
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the capture
cap.release()
cv2.destroyAllWindows()


Face 0: Left Eye Size: 35.23, Right Eye Size: 34.23, Nose Size: 21.02, Mouth Size: 72.34
Face 0: Left Eye Size: 35.13, Right Eye Size: 34.00, Nose Size: 22.00, Mouth Size: 72.03
Face 0: Left Eye Size: 36.06, Right Eye Size: 33.02, Nose Size: 23.09, Mouth Size: 71.06
Face 0: Left Eye Size: 56.04, Right Eye Size: 54.04, Nose Size: 40.20, Mouth Size: 121.07
Face 0: Left Eye Size: 56.04, Right Eye Size: 54.04, Nose Size: 40.20, Mouth Size: 121.07
Face 0: Left Eye Size: 58.03, Right Eye Size: 59.08, Nose Size: 42.05, Mouth Size: 132.31
Face 0: Left Eye Size: 59.03, Right Eye Size: 62.01, Nose Size: 41.01, Mouth Size: 129.25
Face 0: Left Eye Size: 58.03, Right Eye Size: 58.03, Nose Size: 41.05, Mouth Size: 115.11
Face 0: Left Eye Size: 58.03, Right Eye Size: 59.03, Nose Size: 43.10, Mouth Size: 115.11
Face 0: Left Eye Size: 58.08, Right Eye Size: 59.03, Nose Size: 41.05, Mouth Size: 128.14
Face 0: Left Eye Size: 57.01, Right Eye Size: 56.14, Nose Size: 41.19, Mouth Size: 129.14
Face 0: Left 

Face 0: Left Eye Size: 62.07, Right Eye Size: 57.04, Nose Size: 39.32, Mouth Size: 127.00
Face 0: Left Eye Size: 63.07, Right Eye Size: 57.01, Nose Size: 39.32, Mouth Size: 121.04
Face 0: Left Eye Size: 62.03, Right Eye Size: 58.01, Nose Size: 41.30, Mouth Size: 125.00
Face 0: Left Eye Size: 63.07, Right Eye Size: 58.03, Nose Size: 40.61, Mouth Size: 125.00
Face 0: Left Eye Size: 62.07, Right Eye Size: 58.00, Nose Size: 39.46, Mouth Size: 131.00
Face 0: Left Eye Size: 61.03, Right Eye Size: 59.08, Nose Size: 40.45, Mouth Size: 129.10
Face 0: Left Eye Size: 61.07, Right Eye Size: 56.00, Nose Size: 36.35, Mouth Size: 125.14
Face 0: Left Eye Size: 60.03, Right Eye Size: 54.01, Nose Size: 38.47, Mouth Size: 137.13
Face 0: Left Eye Size: 61.07, Right Eye Size: 58.01, Nose Size: 39.32, Mouth Size: 143.17
Face 0: Left Eye Size: 62.07, Right Eye Size: 58.01, Nose Size: 36.35, Mouth Size: 135.09
Face 0: Left Eye Size: 62.07, Right Eye Size: 56.04, Nose Size: 37.48, Mouth Size: 131.38
Face 0: Le

Face 0: Left Eye Size: 59.00, Right Eye Size: 54.08, Nose Size: 42.43, Mouth Size: 121.02
Face 0: Left Eye Size: 57.00, Right Eye Size: 53.08, Nose Size: 43.42, Mouth Size: 128.04
Face 0: Left Eye Size: 58.00, Right Eye Size: 53.04, Nose Size: 42.43, Mouth Size: 125.02
Face 0: Left Eye Size: 54.01, Right Eye Size: 48.17, Nose Size: 42.76, Mouth Size: 122.07
Face 0: Left Eye Size: 57.08, Right Eye Size: 53.04, Nose Size: 41.59, Mouth Size: 129.03
Face 0: Left Eye Size: 57.00, Right Eye Size: 51.09, Nose Size: 44.55, Mouth Size: 126.06
Face 0: Left Eye Size: 59.03, Right Eye Size: 54.08, Nose Size: 41.59, Mouth Size: 124.06
Face 0: Left Eye Size: 53.00, Right Eye Size: 47.04, Nose Size: 41.59, Mouth Size: 119.02
Face 0: Left Eye Size: 55.00, Right Eye Size: 51.09, Nose Size: 40.45, Mouth Size: 127.00
Face 0: Left Eye Size: 57.00, Right Eye Size: 54.08, Nose Size: 42.58, Mouth Size: 134.06
Face 0: Left Eye Size: 60.03, Right Eye Size: 57.01, Nose Size: 40.20, Mouth Size: 133.00
Face 0: Le

Face 0: Left Eye Size: 60.03, Right Eye Size: 57.01, Nose Size: 41.19, Mouth Size: 116.11
Face 0: Left Eye Size: 61.01, Right Eye Size: 58.08, Nose Size: 41.30, Mouth Size: 118.04
Face 0: Left Eye Size: 58.01, Right Eye Size: 49.16, Nose Size: 31.14, Mouth Size: 117.72
Face 0: Left Eye Size: 54.04, Right Eye Size: 47.10, Nose Size: 33.24, Mouth Size: 111.88
Face 0: Left Eye Size: 57.04, Right Eye Size: 49.37, Nose Size: 29.15, Mouth Size: 120.67
Face 0: Left Eye Size: 57.00, Right Eye Size: 49.09, Nose Size: 30.27, Mouth Size: 118.09
Face 0: Left Eye Size: 58.00, Right Eye Size: 48.37, Nose Size: 30.27, Mouth Size: 120.21
Face 0: Left Eye Size: 56.00, Right Eye Size: 49.09, Nose Size: 31.14, Mouth Size: 120.70
Face 0: Left Eye Size: 56.00, Right Eye Size: 49.09, Nose Size: 32.14, Mouth Size: 116.73
Face 0: Left Eye Size: 63.07, Right Eye Size: 55.23, Nose Size: 33.14, Mouth Size: 120.21
Face 0: Left Eye Size: 63.07, Right Eye Size: 61.01, Nose Size: 41.11, Mouth Size: 115.11
Face 0: Le

Face 0: Left Eye Size: 59.01, Right Eye Size: 55.01, Nose Size: 38.05, Mouth Size: 117.15
Face 0: Left Eye Size: 59.01, Right Eye Size: 53.00, Nose Size: 35.36, Mouth Size: 108.37
Face 0: Left Eye Size: 57.00, Right Eye Size: 51.04, Nose Size: 35.23, Mouth Size: 113.28
Face 0: Left Eye Size: 61.03, Right Eye Size: 56.08, Nose Size: 38.05, Mouth Size: 117.02
Face 0: Left Eye Size: 58.01, Right Eye Size: 53.08, Nose Size: 39.32, Mouth Size: 123.02
Face 0: Left Eye Size: 59.01, Right Eye Size: 55.01, Nose Size: 38.05, Mouth Size: 124.02
Face 0: Left Eye Size: 60.03, Right Eye Size: 55.04, Nose Size: 36.12, Mouth Size: 108.66
Face 0: Left Eye Size: 60.03, Right Eye Size: 54.08, Nose Size: 36.12, Mouth Size: 111.45
Face 0: Left Eye Size: 62.13, Right Eye Size: 55.15, Nose Size: 37.05, Mouth Size: 110.77
Face 0: Left Eye Size: 63.13, Right Eye Size: 56.14, Nose Size: 36.22, Mouth Size: 113.44
Face 0: Left Eye Size: 62.13, Right Eye Size: 54.23, Nose Size: 32.25, Mouth Size: 111.54
Face 0: Le